In [1]:
import random
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import Sequential, activations, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
    BatchNormalization, Activation, SpatialDropout2D, GlobalAveragePooling2D,
    Dense, Dropout, Flatten,
    Conv2D, MaxPooling2D, SeparableConv2D
)
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from albumentations import Compose

# Import project utils scripts
import os
import sys

src_path = os.path.join('../src/')

if src_path not in sys.path:
    sys.path.append(src_path)

from visualizationUtils import plot_confusion_matrix
from statisticsUtils import recall_precision_fscore_from_confusion_matrix
from lossesUtils import categorical_focal_loss
from bandUtils import Band 
from labelsUtils import (
    Label,
    LabelCategory,
    category_from_label,
    categories_from_label_set
)

from rasterUtils import make_dataset_from_raster_files
from regionUtils import (
    vietnam_labels_coordinates,
)
from convNetUtils import (
    images_from_dataset,
    labels_from_dataset,
    cross_validation,
    AUGMENTATIONS,
    ImageMultiOutputSequence,
    train_model,
    evaluate_multi_output_model,
)
from config import *

/home/simon/miniconda3/envs/tb/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
NB_PIXEL_AROUND = 4
EPOCHS = 2000
NB_TESTS = 4

labels = [
    Label.COFFEE,
    Label.DENSE_FOREST,
    Label.RUBBER,
    Label.SEASONAL_AGRICULTURE,
    Label.URBAN,
    Label.WATER,
    Label.OTHER_TREE,
    Label.NATIVE_NO_TREE,
    Label.PEPPER,
    Label.TEA,
    Label.RICE,     
    Label.DECIDUOUS_FOREST,
    Label.PINE_TREES,     
    Label.SHRUBLAND_BUSHLAND,     
    Label.SPARE_TREE,     
    Label.GRASSLAND,     
    Label.SECONDARY_DEGRADED_FOREST,     
    Label.MINE_BARESOIL,
]

labels_names = [label.name for label in labels]
categories = pd.unique([category_from_label(label) for label in labels])

bands = [
    Band.COASTAL_AEROSOL.value, 
    Band.BLUE.value, 
    Band.GREEN.value, 
    Band.RED.value, 
    Band.NIR.value, 
    Band.SWIR1.value, 
    Band.SWIR2.value, 
]

In [3]:
labels_coordinates = vietnam_labels_coordinates()

dataset_args = dict(
    labels = labels,
    raster_paths = [os.path.join(DATA_ROOT_PATH, 'Vietnam_2018_whole_year_collection2/merged.tif')],
    labels_coordinates_list = [labels_coordinates],
    nb_pixel_around = NB_PIXEL_AROUND
)

dataset = make_dataset_from_raster_files(**dataset_args)

np.random.shuffle(dataset)

In [4]:
model_name = "final_model"

model = load_model(
    os.path.join(MODEL_ROOT_PATH, 'multi_output_pretrained_with_reptile_v4.hdf5'),
    custom_objects={
        'categorical_focal_loss_fixed': categorical_focal_loss
    }
)

layers = model.layers

# Add trainable layers before output layers
layers = Dense(128, activation='relu')(model.get_layer('last_pretrained_layer').output)
layers = Dense(128, activation='relu')(layers)
layers = Dropout(0.25)(layers)

# Define output layers
label_output = Dense(len(labels), activation='softmax', name="label")(layers)
category_output = Dense(len(categories), activation='softmax', name="category")(layers)

# Create model
model = Model(inputs=model.input, outputs=[label_output, category_output])

model.compile(optimizer='adam', loss={
    'label': categorical_focal_loss([[.25] * len(labels)]),
    'category': categorical_focal_loss([[.25] * len(categories)])
}, metrics={
    'label': 'accuracy',
    'category': 'accuracy'
})

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 9, 9, 7)]    0                                            
__________________________________________________________________________________________________
rescaling_4 (Rescaling)         (None, 9, 9, 7)      0           input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 9, 9, 7)      28          rescaling_4[0][0]                
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 9, 9, 16)     464         batch_normalization_8[0][0]      
______________________________________________________________________________________________

In [5]:
mean_loss, mean_accuracy, histories, conf_matrix = cross_validation(model, dataset, bands, labels, EPOCHS, NB_TESTS, early_stopping=True, with_model_checkpoint=True, model_name=model_name, categories=categories)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 9, 9, 7)]    0                                            
__________________________________________________________________________________________________
rescaling_4 (Rescaling)         (None, 9, 9, 7)      0           input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 9, 9, 7)      28          rescaling_4[0][0]                
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 9, 9, 16)     464         batch_normalization_8[0][0]      
______________________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
conf_matrices, accuracy, loss = evaluate_multi_output_model(trained_model, X_test, Y_test, y_test)

In [ ]:
pl.figure(figsize=(12, 10), dpi=80)

plot_confusion_matrix(conf_matrices[0], labels)

In [ ]:
recall, precision, fscore = recall_precision_fscore_from_confusion_matrix(conf_matrices[0])

print("Precisions: ", precision)
print("Mean precision: ", np.mean(precision))
print("\nRecalls: ", recall)
print("Mean recall: ", np.mean(recall))
print("\nF-Score: ", fscore)
print("Mean f-score: ", np.mean(fscore))

In [ ]:
pl.figure(figsize=(8, 6), dpi=80)

plot_confusion_matrix(conf_matrices[1], [category.name for category in categories])

In [ ]:
recall, precision, fscore = recall_precision_fscore_from_confusion_matrix(conf_matrices[1])

print("Precisions: ", precision)
print("Mean precision: ", np.mean(precision))
print("\nRecalls: ", recall)
print("Mean recall: ", np.mean(recall))
print("\nF-Score: ", fscore)
print("Mean f-score: ", np.mean(fscore))

In [ ]:
figure, axis = pl.subplots(1, 3, figsize=(12,4))
axis[0].plot(history.history['loss'], label='Loss', color="blue")
axis[0].legend()
axis[1].plot(history.history['label_accuracy'], label='Label accuracy', color="blue")
axis[1].set_ylim([0,1])
axis[1].legend()
axis[2].plot(history.history['category_accuracy'], label='Category accuracy', color="blue")
axis[2].set_ylim([0,1])
axis[2].legend()